# Problem 4

In [48]:
import numpy as np
from math import log

In [56]:
freq_table = np.array([
# en      pl       sp   
[0.08167, 0.10503, 0.11525], # a
[0.12702, 0.07352, 0.12181], # e
[0.06966, 0.08328, 0.06247], # i
[0.07507, 0.02445, 0.08683], # o
[0.02758, 0.02062, 0.02927], # u
[0.01974, 0.03206, 0.01008], # y
[0.59926, 0.66104, 0.57429]])# other
lang_probs = np.array([
# en pl sp  
.3, .3, .3])

languages = ['en', 'pl', 'sp']

def lang2ind(lang):
    return languages.index(lang)

def letter2ind(letter):
    vowels = 'aeiouy'
    i = vowels.find(letter)
    if i == -1: # other
        i = len(vowels)
        assert i == 6
    return i
        
def letter_lang_prob(letter, lang):
    return freq_table[letter2ind(letter), lang2ind(lang)]

In [60]:
def string_lang_prob(string, lang, log_prob=True):
    string = np.array(list(string))
    (letters, counts) = np.unique(string, return_counts=True)
    prob = lang_probs[lang2ind(lang)]
    if log_prob:
        prob = log(prob)
    for (l,c) in zip(letters, counts):
        p = letter_lang_prob(l, lang)
        if log_prob:
            prob += c*log(p)
        else:
            prob *= p**c
    return prob
        

In [61]:
def classify_string(string):
    probs = [ string_lang_prob(string, lang) for lang in languages ]
    ind = np.array(probs).argmax()
    print(probs, end=' ')
    return languages[ind]

In [62]:
samples = [
    'bull',
    'burro',
    'kurczak',
    'pollo',
    'litwo, ojczyzno moja, ty jesteś jak zdrowie',
    'dinero',
    'mama just killed a man put a gun against his head',
    'maradona es mas grande que pele'
]
for s in samples:
    print(s, ':')
    l = classify_string(s)
    print('result =', l)

bull :
[-6.3308163657853296, -6.3272893844868836, -6.399027333901266] result = pl
burro :
[-8.9201506331825549, -10.03841444754814, -8.8428304288108475] result = sp
kurczak :
[-9.8600043439588934, -9.4086804930127528, -9.6689204985379682] result = pl
pollo :
[-7.9188204939628015, -9.8680457102160712, -7.7554413454420894] result = sp
litwo, ojczyzno moja, ty jesteś jak zdrowie :
[-53.37921974654337, -55.952882877170573, -54.884714385730781] result = en
dinero :
[-10.057025965217166, -11.252665302764944, -10.189999913279435] result = en
mama just killed a man put a gun against his head :
[-61.033693551686511, -57.213614991625064, -59.91504014324137] result = pl
maradona es mas grande que pele :
[-39.955502456771718, -40.980004554386255, -39.046485080939703] result = sp


# Problem 6

In [63]:
# plot bernoulli

# Problem 7

In [64]:
# rock, paper scissors machine